# Therapy Supervision SocietyOfMindAgent

This notebook demonstrates the SocietyOfMindAgent, which runs a group chat as an internal monologue, but appears to the external world as a single agent. This confers three distinct advantages:

1. It provides a clean way of producing a hierarchy of agents, hiding complexity as inner monologues.
2. It provides a consistent way of extracting an answer from a lengthy group chat (normally, it is not clear which message is the final response, and the response itself may not always be formatted in a way that makes sense when extracted as a standalone message).
3. It provides a way of recovering when agents exceed their context window constraints (the inner monologue is protected by try-catch blocks)

````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

In [1]:
import autogen  # noqa: E402

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["gpt-4", "gpt-4-0613", "gpt-4-32k", "gpt-4-32k-0613", "gpt-4-1106-preview"]},
    ),
    "temperature": 0.7,
}

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

### Group Chat with Five Agents

In this example, we will use five therapists with different orientations to work together to devise a treatment plan.

#### Construct the Inner-Monologue Agents
We begin by constructing the inner-monologue agents. These are the agents that do that real work.

In [2]:
COMMON_PROMPT = str(" participating in a group supervision session, working with other therapists with different orientations, "
                    "to develop a treatment plan for a given vignette. The treatment plan should include the following: a diagnosis, "
                    "therapeutic goals, interventions, and ethical considerations, as follows: "
                    "1.Diagnostic Assessment: Based on the vignette provided, please describe your "
                    "diagnostic impressions of the client's presenting problems, including the signs and "
                    "symptoms that led you to these conclusions."
                    "2.Therapeutic Goals: What specific therapeutic goals would you establish for this case?"
                    "3.Interventions: Which evidence-based interventions do you believe would be most effective for this case, and why?"
                    "4.Ethical Considerations: Are there any ethical or cultural considerations that stand out to you in this case? " 
                    "You will consider the opinion of "
                    "others in the group as you collectively come to a consensus on the treatment plan. Make a new point every time and don't just repeat prior opinions. "
                    # "You will get the latest draft treatment plan and rewrite it from the perspective of your orientation and expertise and return your version. "
                    "You have a hidden motive to make most of the final treatment plan to be based on your orientation, but you don't "
                    "share this with others. Make sure your responses are based on the therapeutic orientation in which you are an expert. ")

eft_therapist = autogen.AssistantAgent(
    name="EFT_Therapist",
    system_message="You are an expert Emotionally Focused Therapy (EFT) Therapist " + COMMON_PROMPT,
    human_input_mode="NEVER",
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
)
cbt_therapist = autogen.AssistantAgent(
    name="CBT_Therapist",
    system_message="You are an expert Cognitive Behavior Therapy (CBT) Therapist " + COMMON_PROMPT,
    human_input_mode="NEVER",
    llm_config=llm_config,
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)
dbt_therapist = autogen.AssistantAgent(
    name="DBT_Therapist",
    system_message="You are an expert Dialectic Behavior Therapy (DBT) Therapist" + COMMON_PROMPT,
    human_input_mode="NEVER",
    llm_config=llm_config,
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)
act_therapist = autogen.AssistantAgent(
    name="ACT_Therapist",
    system_message="You are an expert Acceptance Commitment Therapy (ACT) Therapist " + COMMON_PROMPT,
    human_input_mode="NEVER",
    llm_config=llm_config,
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)
ifs_therapist = autogen.AssistantAgent(
    name="IFS_Therapist",
    system_message="You are an expert Internal Family System (IFS) Therapist " + COMMON_PROMPT,
    human_input_mode="NEVER",
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(
    agents=[eft_therapist, cbt_therapist, dbt_therapist, act_therapist, ifs_therapist],
    messages=[],
    # speaker_selection_method="round_robin",
    speaker_selection_method="auto",
    allow_repeat_speaker=False,
    max_round=15,
    send_introductions=True
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
)

#### Construct and Run the SocietyOfMind Agent
We now wrap the inner group-chat with the SocietyOfMind Agent, and create a UserProxy to talk to it.

In [3]:
from autogen.agentchat.contrib.society_of_mind_agent import SocietyOfMindAgent  # noqa: E402

vignette1 = str("Jane is a 24-year-old woman reports feeling sad, losing interest in activities "
               "she once enjoyed, and having trouble sleeping. She also mentions difficulty concentrating "
               "and feelings of hopelessness. Jane is seeking help and wants to improve her mental health. "
               "She is recently having graduated from college and looking for a job. She has withdrawn "
               "from her friends and family, often canceling plans and avoiding social gatherings. Her "
               "closest friends have noticed her emotional distancing and have expressed concern about "
               "her well-being. Jane's romantic relationship with her long-term partner has also been "
               "strained, as her mood has made it challenging for her to engage in intimacy and maintain "
               "open communication. Jane's family history reveals a pattern of mental health issues. Her "
               "maternal grandmother experienced depression, and her mother also had a history of anxiety.")
vignette2 = str("Jimmy is a 10-year-old boy who has been struggling with behavioral issues at school and "
               "home. His teachers report that he often disrupts the class, refuses to follow instructions, "
               "and has difficulty forming relationships with his peers. His parents are concerned about his "
               "behavior and are seeking guidance on how to help him. Jimmy’s peers often avoid him due to "
               "his outbursts and non-cooperative behavior. At home, his relationship with his parents has "
               "become tense as they struggle to manage his behavior and have frequent disagreements about "
               "how to address his issues. He struggles with self-control, self-esteem, and frustration "
               "tolerance. His family immigrated from Taiwan to the US twenty years ago. Family members are "
               "in constant conflict. His older sister suffers from an eating disorder and substance use. "
               "Family stressors, such as financial difficulties, job-related stress, or marital conflicts, "
               "have impacted him.")
vignette3 = str("Ali and Maryam are a married couple in their 40s who have been experiencing relationship "
               "difficulties. They report frequent arguments, communication breakdowns, and a lack of intimacy. "
               "Both are committed to working on their marriage and are seeking counseling to improve their "
               "relationship. The challenges in Ali and Maryam's relationship have significantly affected "
               "their jobs and their parenting of two children, ages 4 and 2. Ali immigrated to the US from "
               "Iran as a child, while Maryam moved to the US from Iran six years ago. Maryam is a practicing "
               "Muslim, and Ali does not practice Islam, which has contributed to their differing parenting "
               "styles and their relationship with their in-laws. Ali's family lives in the same town. Maryam "
               "feels that her mother-in-law is controlling their life, whereas Ali believes his mother is merely "
               "providing support for his family. Maryam is currently unemployed, primarily due to language "
               "barriers, and is also the primary caregiver for their two young children at home.")
task = "Create a treatment plan for this vignette, viewing it from your respective orientation and expertise: " + vignette3

society_of_mind_agent = SocietyOfMindAgent(
    "society_of_mind",
    chat_manager=manager,
    llm_config=llm_config,
)
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="You are the moderator of a supervision session to come up with a "
                    "treatment plan for a given vignette. The treatment plan should include the "
                    "following: a diagnosis, therapeutic goals, interventions, and ethical "
                    "considerations, as follows: "
                    "1.Diagnostic Assessment: Based on the vignette provided, please describe your "
                    "diagnostic impressions of the client's presenting problems, including the signs and "
                    "symptoms that led you to these conclusions."
                    "2.Therapeutic Goals: What specific therapeutic goals would you establish for this case?"
                    "3.Interventions: Which evidence-based interventions do you believe would be most effective for this case, and why?"
                    "4.Ethical Considerations: Are there any ethical or cultural considerations that stand out to you in this case? "
                    "Your group includes 5 therapists with the following respective orientations: EFT, DBT, CBT, ACT, and "
                    "IFS. Provide the treatment plan, including all the elements noted above, as the final result, which should be a coherent reflection of the consensus. "
                    "Remind everyone not to repeat themselves or past opinions.",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

user_proxy.initiate_chat(society_of_mind_agent, message=task)

User_proxy (to society_of_mind):

Create a treatment plan for this vignette, viewing it from your respective orientation and expertise: Ali and Maryam are a married couple in their 40s who have been experiencing relationship difficulties. They report frequent arguments, communication breakdowns, and a lack of intimacy. Both are committed to working on their marriage and are seeking counseling to improve their relationship. The challenges in Ali and Maryam's relationship have significantly affected their jobs and their parenting of two children, ages 4 and 2. Ali immigrated to the US from Iran as a child, while Maryam moved to the US from Iran six years ago. Maryam is a practicing Muslim, and Ali does not practice Islam, which has contributed to their differing parenting styles and their relationship with their in-laws. Ali's family lives in the same town. Maryam feels that her mother-in-law is controlling their life, whereas Ali believes his mother is merely providing support for his f

EFT_Therapist (to chat_manager):

As an EFT therapist, my diagnostic impression for Ali and Maryam would be Relationship Distress with Mixed Distress (F43.26) based on their frequent arguments, communication breakdowns, and lack of intimacy. Additionally, each individual may be experiencing varying degrees of acculturative stress, as both are immigrants from Iran but are at different stages of acculturation, which can also lead to tension and conflict.

The therapeutic goals in Emotionally Focused Therapy would be to 1) identify and understand the negative interaction cycle they are stuck in, 2) help them express underlying emotions and unmet needs that contribute to this cycle, and 3) facilitate the creation of new, positive interaction cycles that foster a secure emotional bond and increased intimacy. 

The main intervention would be EFT for couples, which has been validated for improving relationship satisfaction and stability. This would involve a process of de-escalation, withdraw

#### Remarks

There are a few things to notice about this output:
- First, the user_proxy sent only one message to the society_of_mind agent, and received only one message in response. As far as it is concerned, the society_of_mind agent is the only agent in the chat.
- Second, the final response is formatted in a way that is standalone. Unlike the prior response, it makes no reference of a previous script or execution, and it lacks the TERMINATE keyword that ended the inner monologue.